In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { min-width:1500px !important; }</style>"))

import os
import sys
sys.path.append('..')

import time
import datetime

import importlib

import tensorflow as tf
from tensorflow import keras

import src
from src.data import cifar100
from src.models import basic, factory
from src.meta.meta import Metadata
from src.utils import debug_utils, file_utils, io_utils

print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S   '), tf.__name__, '-', tf.__version__, sep='')

/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

2019-08-14 11:35:50   tensorflow-1.14.0


/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([

In [2]:
from src.utils.logger import Logging
logger = Logging.get_logger(__name__)
Logging.attach_stdout()

# Setup

In [3]:
_NAME = basic.NAME
_EPOCH = 10

filepath_md = file_utils.model_filepath_md(_NAME, _EPOCH)
print('filepath_md:            ', filepath_md)
filepath_pb = file_utils.model_filepath_pb(_NAME, _EPOCH)
print('filepath_pb:            ', filepath_pb)
filepath_weights_h5 = file_utils.model_filepath_weights_h5(_NAME, _EPOCH)
print('filepath_weights_h5:    ', filepath_weights_h5)
filepath_tflite = file_utils.model_filepath_tflite(_NAME, _EPOCH)
print('filepath_tflite:        ', filepath_tflite)
filepath_edgetpu_tflite = file_utils.model_filepath_edgetpu_tflite(_NAME, _EPOCH)
print('filepath_edgetpu_tflite:', filepath_edgetpu_tflite)

output_data_file = 'tmp/test_data_%s.csv' % datetime.datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
print('output_data_file:       ', output_data_file)

TRIALS = 10
TEST_SIZES = [1, 10, 100, 1000]
REPEAT = 2

filepath_md:             /space/code/deep-learning/models/basic/010/basic_010.md
filepath_pb:             /space/code/deep-learning/models/basic/010/basic_010.pb
filepath_weights_h5:     /space/code/deep-learning/models/basic/010/basic_010_weights.h5
filepath_tflite:         /space/code/deep-learning/models/basic/010/basic_010_int8.tflite
filepath_edgetpu_tflite: /space/code/deep-learning/models/basic/010/basic_010_int8_edgetpu.tflite
output_data_file:        tmp/test_data_2019-08-14-11:35:51.csv


In [4]:
%%bash

find ../models | grep basic | grep -E '(010|009)' | sort

../models/basic/009
../models/basic/009/basic_009_checkpoint.h5
../models/basic/009/basic_009.h5
../models/basic/009/basic_009.md
../models/basic/009/basic_009_weights.h5
../models/basic/010
../models/basic/010/basic_010_checkpoint.h5
../models/basic/010/basic_010.h5
../models/basic/010/basic_010_int8_edgetpu.log
../models/basic/010/basic_010_int8_edgetpu.tflite
../models/basic/010/basic_010_int8.tflite
../models/basic/010/basic_010.md
../models/basic/010/basic_010.pb
../models/basic/010/basic_010_weights.h5


# Data

In [5]:
from src.data import cifar100

In [6]:
(TRAIN_X, TRAIN_Y), (TEST_X, TEST_Y) = cifar100.load_data(normalize=True)

# Keras

In [7]:
from src.keras.model import KerasModel
importlib.reload(src.keras.model)
from src.keras.model import KerasModel

In [8]:
USE_EXISTING = False
if USE_EXISTING:
    ret, keras_model = KerasModel.from_h5(filepath_md, '../resnet50_csv_16.h5')
else:
    f_construct_keras_model = factory.get_model_create_func(_NAME, cifar100.INPUT_SHAPE, len(cifar100.CLASSES))
    ret, keras_model = KerasModel.from_weights_h5(filepath_md, f_construct_keras_model, filepath_weights_h5)   
assert ret == 0

2019-08-14 11:35:53,174 DEBUG [src.keras.model:92] (MainThread) [basic|10] Constructing keras model from factory function...


W0814 11:35:53.203348 140453918638208 deprecation.py:506] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2019-08-14 11:35:53,219 DEBUG [src.keras.model:97] (MainThread) [basic|10] Loading keras model weights from /space/code/deep-learning/models/basic/010/basic_010_weights.h5...
2019-08-14 11:35:53,822 DEBUG [src.keras.model:39] (MainThread) [basic|10|keras] Setting metadata input names to ['basic0:0']
2019-08-14 11:35:53,823 DEBUG [src.keras.model:47] (MainThread) [basic|10|keras] Setting metadata out names to ['basic2/Softmax:0']


# freeze_graph

In [9]:
from tensorflow.core.protobuf import saver_pb2
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph
from tensorflow.python.training import saver as saver_lib

In [10]:
def keras2pb(model, workingdir, path):
    # get output names
    out_names = [out.op.name for out in model.outputs]
    out_names = ','.join(out_names)
    print('out_names:', out_names)
    
    # learning phase 0
    keras.backend.set_learning_phase(0)
    sess = keras.backend.get_session()
    
    # checkpoints
    print('Saving checkpoints...')
    saver = saver_lib.Saver(write_version=saver_pb2.SaverDef.V2)
    checkpoint_path = saver.save(sess, workingdir + '/saved_ckpt',
                                 global_step=0, latest_filename='checkpoint_state')
    
    # write graph
    print('Writing graph to ', workingdir, '/tmp.pb', sep='')
    graph_io.write_graph(sess.graph, workingdir, 'tmp.pb')
    
    # freeze graph
    filename = workingdir + '/tmp.pb'
    print('Freezing graph to ', filename, '...', sep='')
    freeze_graph.freeze_graph(filename, '', False, checkpoint_path, out_names,
                              "save/restore_all", "save/Const:0", path, False, "")

In [11]:
keras2pb(keras_model.keras_model, 'tmp', 'output.pb')

W0814 11:35:55.139091 140453918638208 deprecation.py:323] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/tools/freeze_graph.py:127: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


out_names: basic2/Softmax
Saving checkpoints...
Writing graph to tmp/tmp.pb
Freezing graph to tmp/tmp.pb...


IndexError: list index out of range

In [13]:
keras.backend.clear_session()
del keras_model.keras_model
del keras_model
keras.backend.clear_session()

2019-08-14 11:26:12,272 INFO  [src.keras.model:63] (MainThread) basic Closing session...


# Debug

In [ ]:
help(freeze_graph.freeze_graph)

In [ ]:
from src.tf import utils
importlib.reload(src.tf.utils)

In [ ]:
help(utils)

In [ ]:
utils.devices()

In [ ]:
utils.get_available_gpus()

In [ ]:
utils.is_gpu_available()


In [ ]:
utils.gpu_device_name()
